### Purpose: In order to reduce computational expense, remove rows with <= 5 (Pos/Neg) points

In [40]:
# Import required Libraries
# !pip install fastparquet
import math
import pandas_datareader as web
import numpy as np 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [41]:
full_parquet = pd.read_parquet('wsb.parquet.gz', engine="fastparquet")
full_parquet = full_parquet.set_index('created_utc')

In [42]:
t = full_parquet.score.abs() > 5
full_parquet = full_parquet[t]
# t = full_parquet.gilded > 1
full_parquet

,score,gilded,wordcount,0,1,2,3,4,5,6,...,18,19,20,21,22,23,24,DATE,OPrc,CPrc
created_utc,,,,,,,,,,,,,,,,,,,,,
2017-01-03 00:02:01,10,0,9,-0.050337,0.131659,-0.050066,-0.036513,-0.016233,-0.029470,0.120017,...,0.047896,0.083995,0.083635,-0.004959,0.003192,0.035591,0.011367,2017-01-03,225.07,225.24
2017-01-03 00:04:36,10,0,29,0.016263,0.105922,0.004202,-0.049164,-0.050180,-0.007121,0.172484,...,-0.047003,-0.050088,-0.015005,-0.024337,0.041875,0.040734,-0.040855,2017-01-03,225.07,225.24
2017-01-03 00:05:54,37,0,8,0.080844,0.107275,0.043767,-0.035451,-0.003215,-0.109981,0.233787,...,-0.050461,-0.053820,-0.007516,0.045674,0.034464,0.062269,-0.047652,2017-01-03,225.07,225.24
2017-01-03 00:10:15,12,0,2,-0.049617,0.259752,-0.031981,-0.037760,-0.129099,0.034867,0.196902,...,-0.137091,0.042870,0.062167,0.125428,0.035338,0.204555,-0.054292,2017-01-03,225.07,225.24
2017-01-03 00:10:45,94,0,9,0.013466,0.118551,-0.038913,-0.041602,-0.133586,-0.057393,0.223735,...,0.020307,-0.101664,0.071954,0.053199,0.013205,0.107264,-0.070397,2017-01-03,225.07,225.24
2017-01-03 00:14:06,14,0,33,-0.008503,0.072983,-0.057180,-0.000776,-0.017979,-0.019481,0.177660,...,-0.023432,-0.018831,-0.040776,-0.039624,0.024249,0.004514,-0.070760,2017-01-03,225.07,225.24
2017-01-03 00:15:43,13,0,57,-0.024865,0.080602,-0.024598,-0.011565,-0.080335,0.027629,0.187195,...,-0.002321,-0.015897,0.014804,-0.001310,0.076481,0.004833,-0.075602,2017-01-03,225.07,225.24
2017-01-03 00:16:06,17,0,3,-0.020739,0.100826,-0.229980,0.017683,-0.125350,-0.170245,0.076074,...,0.187350,-0.082510,0.102417,0.067108,-0.208255,-0.113719,-0.115839,2017-01-03,225.07,225.24
2017-01-03 00:23:10,-15,0,19,0.028589,0.091381,-0.041042,-0.025330,0.027237,-0.051487,0.123717,...,0.008512,-0.054332,-0.030036,-0.031817,0.010364,-0.010013,-0.113816,2017-01-03,225.07,225.24


In [43]:
# 
full_parquet.drop(['DATE','OPrc','CPrc'], axis=1, inplace=True)
full_parquet.head()

,score,gilded,wordcount,0,1,2,3,4,5,6,...,15,16,17,18,19,20,21,22,23,24
created_utc,,,,,,,,,,,,,,,,,,,,,
2017-01-03 00:02:01,10,0,9,-0.050337,0.131659,-0.050066,-0.036513,-0.016233,-0.029470,0.120017,...,0.019922,0.080937,-0.020296,0.047896,0.083995,0.083635,-0.004959,0.003192,0.035591,0.011367
2017-01-03 00:04:36,10,0,29,0.016263,0.105922,0.004202,-0.049164,-0.050180,-0.007121,0.172484,...,0.067639,0.089193,-0.038352,-0.047003,-0.050088,-0.015005,-0.024337,0.041875,0.040734,-0.040855
2017-01-03 00:05:54,37,0,8,0.080844,0.107275,0.043767,-0.035451,-0.003215,-0.109981,0.233787,...,0.065161,0.080715,-0.120598,-0.050461,-0.053820,-0.007516,0.045674,0.034464,0.062269,-0.047652
2017-01-03 00:10:15,12,0,2,-0.049617,0.259752,-0.031981,-0.037760,-0.129099,0.034867,0.196902,...,-0.063569,0.026807,-0.122583,-0.137091,0.042870,0.062167,0.125428,0.035338,0.204555,-0.054292
2017-01-03 00:10:45,94,0,9,0.013466,0.118551,-0.038913,-0.041602,-0.133586,-0.057393,0.223735,...,0.024832,0.066150,-0.108282,0.020307,-0.101664,0.071954,0.053199,0.013205,0.107264,-0.070397


In [44]:
tmp = full_parquet.resample('H').median()
tmp.head()

,score,gilded,wordcount,0,1,2,3,4,5,6,...,15,16,17,18,19,20,21,22,23,24
created_utc,,,,,,,,,,,,,,,,,,,,,
2017-01-03 00:00:00,12.5,0.0,10.5,-0.022802,0.092325,-0.026620,-0.027104,-0.055260,-0.023351,0.146637,...,0.016766,0.060790,-0.044855,-0.009599,-0.051954,0.013111,0.029598,0.025982,0.033215,-0.068109
2017-01-03 01:00:00,11.0,0.0,14.0,-0.006141,0.072526,-0.012687,-0.019816,-0.038183,-0.014714,0.175447,...,0.020134,0.039672,-0.025471,-0.011119,-0.017077,0.006845,0.016514,0.031367,0.015312,-0.058244
2017-01-03 02:00:00,14.0,0.0,13.0,-0.001822,0.078060,-0.029167,-0.022511,-0.037916,-0.031798,0.157538,...,0.025561,0.048476,-0.039750,0.010611,-0.001064,-0.001501,0.024138,0.039381,0.001204,-0.050443
2017-01-03 03:00:00,18.0,0.0,8.0,-0.008053,0.096946,0.001085,-0.002828,-0.055145,-0.017066,0.148256,...,0.022485,0.042912,-0.050658,0.015111,-0.017721,0.018283,0.053949,0.026359,0.017344,-0.031969
2017-01-03 04:00:00,8.5,0.0,15.5,-0.019297,0.089258,-0.022849,-0.013116,-0.043810,-0.013855,0.143886,...,0.004803,0.057398,-0.046153,0.002006,-0.029689,-0.033309,0.003705,0.014602,0.016177,-0.061811


In [45]:
# tmp = full_parquet.resample('H').mean()
# tmp.head()

In [46]:
tmp.to_parquet('WSB.hourly.parquet.filtered.median.min.score.5.gzip', compression='gzip')  